In [25]:
import random
import string
from pathlib import Path
import os
import requests
import shutil

from git import Repo
from bs4 import BeautifulSoup as Soup
import openai
from PIL import Image

In [26]:
#PATH_TO_BLOG_REPO = Path("/Users/marci/GIT/OpenAI-API-Python-Course/04-Auto-Blog-Post/b4shy.github.io/.git/")
PATH_TO_BLOG_REPO = Path('c:\\Users\\derek\\Documents\\GitHub\\DerekLW6.github.io\\.git')
PATH_TO_BLOG = PATH_TO_BLOG_REPO.parent
PATH_TO_CONTENT = PATH_TO_BLOG/"content"
PATH_TO_CONTENT.mkdir(exist_ok=True, parents=True)

In [27]:
PATH_TO_BLOG

WindowsPath('c:/Users/derek/Documents/GitHub/DerekLW6.github.io')

In [28]:
def random_string(length):
    letters = string.ascii_lowercase + "\n"
    
    return ''.join(random.choice(letters) for i in range(length))

random_string(10)

'h\nbaaktqmk'

In [29]:
def update_blog(commit_message="Updated blog"):
    repo = Repo(PATH_TO_BLOG_REPO)
    repo.git.add(all=True)
    repo.index.commit(commit_message)
    origin = repo.remote(name='origin')
    origin.push()

In [30]:
random_blogpost = random_string(1000)

In [31]:
with open(PATH_TO_BLOG/"index.html", "w") as f:
    f.write(random_blogpost)

In [32]:
update_blog()

In [33]:
def create_new_blog(title, content, cover_image=Path("test.jpg")):
    
    cover_image = Path(cover_image)
    
    files =len(list(PATH_TO_CONTENT.glob("*.html")))
    new_title = f"{files+1}.html"
    path_to_new_content = PATH_TO_CONTENT/new_title
    
    shutil.copy(cover_image, PATH_TO_CONTENT)
    if not os.path.exists(path_to_new_content):
        with open(path_to_new_content, "w") as f:
            f.write("<!DOCTYPE html>\n")
            f.write("<html>\n")
            f.write("<head>\n")
            f.write(f"<title> {title} </title>\n")
            f.write("</head>\n")
            
            f.write("<body>\n")
            f.write(f"<img src='{cover_image.name}' alt='Cover Image'> <br />\n")
            f.write(f"<h1> {title} </h1>")
            f.write(content.replace("\n", "<br />\n"))
            f.write("</body>\n")
            f.write("</html>\n")
            print("Blog created")
            return path_to_new_content
    else:
        raise FileExistsError("File already exist! Abort")

In [34]:
path_to_new_content = create_new_blog("Title", random_string(5000))

Blog created


In [35]:
with open(PATH_TO_BLOG/"index.html") as index:
    soup = Soup(index.read())
str(soup)

'<html><body><p>hacinmgtzavnemvijzxusjpbxkvxezfypvjjpvkerfnffprvxjiochhdcnhvreqlnbatdjcfayyn\nqoxxsuruywiqoxohsjkygvgowzt\ndowjhxarrvfcqwsbfwvfrtyiqycizexhslccezvh\nlhskkxfagvnikhfxiinuwzctczfvn\n\nfx\nifdizmpjnoqupqecmmsxmyrusuiqggnkcaclrkudqxztoomim\nktgoyftwrxnpkbaukdxaqbpylfevhdmpyjlktksmuuatryjmz\nuhhgv\nwbqymvgsxrkmbwvtevjslyqrhjjkdoevbvbukypaaqmqtnnrtrjlcrxbcurjgjrfo\nzqbjkisqrgfejmpnwbppehhfmomdmmz\nvneewretlgpbh\notdzltludgptigws\ndftcbllckeoqbomlflppejrneriwtsylajuq\njjjmnnxvinckibpidzopiifggynczejhobjyzqglckwnhcjvesvgqkrxjsdputmjbijszwokwtbkrdbqrksouyvvkkzfmopvpl\nadbdkstyayrqkykncfk\nzwlirhmbcnnjbdyinshlqofjjnsfgshklduqkulfcpvypscsujjkmxucpnlulewhtrxtflhvjyua\nxyhqvvmulqcfrovdzpeqbhlv\nfe\njzufxltjfhxurgfr\nwbifhewhfgmhhscwrqueif\narhubecabtrz\ncaywbxxbcxftydjscjqliqryzmybbeyegyiybuxqgddmmlnbgjonyzbkfj\nhkgjchnbc\nfugt\nrrz\nx\niqikdozfwfbsnfbdqwcqksyzuimtqiatfgevxvbqmbjtlvvqptnvqwauclqmsstpfzvnhthrvzrwdewjhtzeltgalflkgekobrprhhqdyokjezunkiisnslpgmmlgsdcgw\nlhsdogdbvkdiovu\

In [36]:
def check_for_duplicate_links(path_to_new_content, links):
    urls = [str(link.get("href")) for link in links]
    content_path = str(Path(*path_to_new_content.parts[-2:]))
    return content_path in urls
    

In [37]:
def write_to_index(path_to_new_content):
    with open(PATH_TO_BLOG/"index.html") as index:
        soup = Soup(index.read())

    links = soup.find_all("a")
    last_link = links[-1]
    
    if check_for_duplicate_links(path_to_new_content, links):
        raise ValueError("Link does already exist!")
        
    link_to_new_blog = soup.new_tag("a", href=Path(*path_to_new_content.parts[-2:]))
    link_to_new_blog.string = path_to_new_content.name.split(".")[0]
    last_link.insert_after(link_to_new_blog)
    
    with open(PATH_TO_BLOG/"index.html", "w") as f:
        f.write(str(soup.prettify(formatter='html')))

In [38]:
write_to_index(path_to_new_content)

IndexError: list index out of range

In [ ]:
update_blog()

### OpenAI Portion

In [ ]:
# Uncomment below and swap in your key to place your environment key using Python
# Then you can delete the key string and the code cell below will still work!
# os.environ["OPENAI_API_KEY"] = "Your key goes here!

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
def create_prompt(title):
    prompt = """Jose's Website
 
 Biography
 I am a Python instructor teaching people machine learning!


 Blog

 Jan 31, 2023
 Title: Why AI will never replace the radiologist
 tags: tech, machine-learning, radiology
 Summary:  I talk about the cons of machine learning in radiology. I explain why I think that AI will never replace the radiologist.
 Full text: """.format(title)
    return prompt

In [ ]:
title = "Why AI will never replace the radiologist"
print(create_prompt(title))

Jose's Website
 
 Biography
 I am a Python instructor teaching people machine learning!


 Blog

 Jan 31, 2023
 Title: Why AI will never replace the radiologist
 tags: tech, machine-learning, radiology
 Summary:  I talk about the cons of machine learning in radiology. I explain why I think that AI will never replace the radiologist.
 Full text: 


In [23]:
response = openai.Completion.create(engine="text-davinci-003",
                                            prompt=create_prompt(title),
                                            max_tokens=512,  # we increased the tokens to get a longer blog post
                                            temperature=0.7)

In [24]:
blog_content = response["choices"][0]["text"]

In [39]:
print(blog_content)



Artificial intelligence (AI) is becoming increasingly popular in healthcare and one of the most popular applications of AI is in the field of radiology. The idea of using AI to diagnose diseases and detect abnormalities may seem appealing, but it is important to understand that AI is no substitute for the experienced radiologist.

AI may be able to detect certain patterns in images more accurately than the human eye, but it cannot replace the expertise of a radiologist. AI can only work with the data it is given, and it cannot make any judgement calls based on the context of a particular patient. AI can also be easily fooled by subtle changes in images, and its accuracy can be affected by a variety of factors.

AI also requires a large amount of data to work with, and this can be difficult to obtain in the medical field. AI algorithms are also constantly evolving, so they are not able to keep up with changes in the medical field.

At the end of the day, AI is a tool that can be used 

### Calling DALLE-2

In [40]:
def dalle2_prompt(title):
    prompt = f"Pixel art showing '{title}'."
    return prompt

In [41]:
image_prompt = dalle2_prompt(title)
image_prompt

"Pixel art showing 'Why AI will never replace the radiologist'."

In [42]:
response = openai.Image.create(
  prompt=image_prompt,
  n=1,
  size="1024x1024"
)


In [43]:
# Pulling up the image
image_url = response['data'][0]['url']
# image_url

In [46]:
def save_image(image_url, file_name):
    image_res = requests.get(image_url, stream = True)
    
    if image_res.status_code == 200:
        with open(file_name,'wb') as f:
            shutil.copyfileobj(image_res.raw, f)
    else:
        print("Error downloading image!")
    return image_res.status_code


In [47]:
save_path = "title2.png"

In [48]:
save_image(image_url, save_path)

200

In [ ]:
# Image looks really dumb
# Image.open(save_path)

In [50]:
path_to_new_content = create_new_blog(title, blog_content, save_path)

Blog created


In [ ]:
write_to_index(path_to_new_content)

In [52]:
update_blog()